### 載入MNIST手寫數字數據集

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

### 定義模型函式

In [3]:
def weight_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape,stddev = 0.1),name = name)

def bias_variable(shape,name):
    return tf.Variable(tf.constant(0.1,shape = shape),name = name)

### 定義變數

In [5]:
n_steps = 28
n_input = 28
n_hidden = 128
n_classes = 10

X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y_true = tf.placeholder(tf.float32,[None,n_classes])

In [6]:
weight = weight_variable([n_hidden,n_classes],'w1')
bias = bias_variable([n_classes],'b1')

### reshape 輸入

In [8]:
x_transpose = tf.transpose(X,[1,0,2])
x_reshape = tf.reshape(x_transpose,[-1,n_input])
x_split = tf.split(x_reshape,n_steps,0)

### 定義模型

In [11]:
from tensorflow.contrib import rnn

lstm = rnn.BasicLSTMCell(n_hidden,forget_bias = 1.0)
h, states = rnn.static_rnn(lstm,x_split,dtype = tf.float32)

In [13]:
h_fc = tf.matmul(h[-1],weight)+bias

### 損失函數與優化器

In [16]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits = h_fc,labels = y_true)
opt = tf.train.AdamOptimizer(0.01).minimize(loss)

correct_prediction = tf.equal(tf.argmax(h_fc,1),tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

### 實例化執行

In [19]:
import numpy as np

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(20000):
        batchx,batchy = mnist.train.next_batch(100)
        batchx = np.reshape(batchx,[100,n_steps,n_input])
        if step%1000 == 0:
            print("Step:{},Accuracy:{}".format(step,accuracy.eval(feed_dict = {X:batchx,y_true:batchy})))
        
        sess.run(opt,feed_dict = {X:batchx,y_true:batchy})
    
    print("Test Accuracy:{}".format(accuracy.eval(feed_dict = {X:np.reshape(mnist.test.images,[-1,28,28]),y_true:mnist.test.labels})))

Step:0,Accuracy:0.10000000149011612
Step:1000,Accuracy:0.9599999785423279
Step:2000,Accuracy:0.9800000190734863
Step:3000,Accuracy:0.9900000095367432
Step:4000,Accuracy:0.9800000190734863
Step:5000,Accuracy:0.9900000095367432
Step:6000,Accuracy:1.0
Step:7000,Accuracy:0.9800000190734863
Step:8000,Accuracy:1.0
Step:9000,Accuracy:0.9900000095367432
Step:10000,Accuracy:0.9900000095367432
Step:11000,Accuracy:1.0
Step:12000,Accuracy:0.9900000095367432
Step:13000,Accuracy:0.9900000095367432
Step:14000,Accuracy:0.9900000095367432
Step:15000,Accuracy:0.9900000095367432
Step:16000,Accuracy:0.9900000095367432
Step:17000,Accuracy:0.9800000190734863
Step:18000,Accuracy:0.9700000286102295
Step:19000,Accuracy:0.9800000190734863
Test Accuracy:0.9742000102996826
